In [2]:
from pyspark import SparkConf, SparkContext

1. Creamos un contexto nuevo de Spark para leer un archivo de texto

In [3]:
conf = SparkConf().setAppName('readFile')
spark = SparkContext.getOrCreate(conf=conf)

your 131072x1 screen size is bogus. expect trouble


24/02/27 20:50:23 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/02/27 20:50:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/27 20:50:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [110]:
text = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/sampletext.txt")

In [111]:
type(text)

pyspark.rdd.RDD

In [112]:
text.collect()

['1 2 23 4', '10 23 101 67', '45 65 34 56', '101 54 34 101']

2. Sobre el __rdd__ aplicamos una __transformacion__ map que devuelve otro __rdd__

In [113]:
miFile_rdd = text.map(lambda x:x.split(' '))


In [114]:
type(miFile_rdd)

pyspark.rdd.PipelinedRDD

In [115]:
miFile_rdd.collect()

[['1', '2', '23', '4'],
 ['10', '23', '101', '67'],
 ['45', '65', '34', '56'],
 ['101', '54', '34', '101']]

3. En este ejemplo queremos usar una funcion __foo__ definida por el usuario donde, primero hagamos lo mismo que se hizo en __2__ pero luego se castea cada elemento a __int__ y se le suma 2.

In [116]:
def mi_foo(x):
    l =  x.split(' ')
    ls = []
    for num in l:
        ls.append(int(num) + 2)
    return ls

miFile  = text.map(mi_foo)
miFile.collect()

[[3, 4, 25, 6], [12, 25, 103, 69], [47, 67, 36, 58], [103, 56, 36, 103]]

Ejercicio.

Tenemos un archivo con saludos (texto) y debemos devolver un array con la logitud de cada palabra.

In [118]:
fileText = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/wordcount.txt")

In [119]:
fileText.collect()

['Hi how are you?', 'Hope you are doing', 'great']

In [120]:
def foo(x):
    array = x.split(' ')
    l2 = []
    for word in array:
        l2.append(len(word))
    return l2

fileText.map(foo).collect()

[[2, 3, 3, 4], [4, 3, 3, 5], [5]]

Variacion del ejercicio.

En lugar de usar una __UDF__ usamos una funcion __lambda__ con una __list comprehention__ para obtener el mismo resultado.

In [121]:
fileText.map(lambda x: [len(word) for word in x.split(' ')]).collect()

[[2, 3, 3, 4], [4, 3, 3, 5], [5]]

4. FlatMap

Devuelve todos los elementos en un tipo de dato __unificado__  generando un nuevo __rdd__

In [122]:
fileText.flatMap(lambda x:x.split(' ')).collect()

['Hi', 'how', 'are', 'you?', 'Hope', 'you', 'are', 'doing', 'great']

5. filter - lambda

Se usa para segmentar el resultado de una __rdd__

In [123]:
text.filter(lambda x: x in ('10 23 45 67', '87 54 34 101')).collect()

[]

Usar filter sin __lambda function__

In [124]:
def foo(x):
    if x in ('10 23 45 67', '87 54 34 101'):
        return True
    return False


text.filter(foo).collect()

[]

6.1 Ejercicio 2

Construir un filtro que elimine las palabras que comiencen con __a__ o __c__

In [125]:
fileText = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/quiz2.txt")
fileText.collect()

['this mango company animal',
 'cat dog ant mic laptop',
 'chair seitch mobile am charger cover',
 'amanda any alarm ant',
 'este es mi ejemplo']

Con lambda

In [126]:
rdd_rts = fileText.flatMap(lambda x:x.split(' '))\
    .filter(lambda x: x.startswith("a") or x.startswith("c"))
print(rdd_rts.collect())

['company', 'animal', 'cat', 'ant', 'chair', 'am', 'charger', 'cover', 'amanda', 'any', 'alarm', 'ant']


Sin lambda

In [127]:
def foo(x):
    if x.startswith("a") or x.startswith("c"):
        return False
    else:
        return True
    
    
rdd_rts = fileText.flatMap(lambda x:x.split(' ')).filter(foo)
print(rdd_rts.collect())  

['this', 'mango', 'dog', 'mic', 'laptop', 'seitch', 'mobile', 'este', 'es', 'mi', 'ejemplo']


7. RDD Distinct

In [128]:
rdd_text = text.flatMap(lambda x:x.split(' ')).distinct()
print(rdd_text.collect())

['1', '4', '10', '45', '34', '56', '54', '2', '23', '101', '67', '65']


In [129]:
text.map(lambda x: x.split(' ')).collect()

[['1', '2', '23', '4'],
 ['10', '23', '101', '67'],
 ['45', '65', '34', '56'],
 ['101', '54', '34', '101']]

8. GroupByKey

In [138]:
rdd_split = text.flatMap(lambda x: x.split(' '))
rdd_key_val = rdd_split.map(lambda x: (x, 1))
rdd_reduce = rdd_key_val.groupByKey()
rdd_mapVal = rdd_reduce.mapValues(list).collect()
print(rdd_mapVal)

[('1', [1]), ('4', [1]), ('10', [1]), ('45', [1]), ('34', [1, 1]), ('56', [1]), ('54', [1]), ('2', [1]), ('23', [1, 1]), ('101', [1, 1, 1]), ('67', [1]), ('65', [1])]


9. reduceByKey

Similar al GroupByKey pero devuelve un __rdd__ agrupado con una __operacion__ sobre los valores de las __keys__

In [144]:
rdd_split = text.flatMap(lambda x:x.split(' '))
rdd_map = rdd_split.map(lambda x: (x, 1))
rdd_reduce = rdd_map.reduceByKey(lambda x,y: x + y)
print(rdd_reduce.collect())

[('1', 1), ('4', 1), ('10', 1), ('45', 1), ('34', 2), ('56', 1), ('54', 1), ('2', 1), ('23', 2), ('101', 3), ('67', 1), ('65', 1)]


10. Ejercicio 

Escribir un script que devuelva la cantidad de cada palabra contenida en el archivo.

In [145]:
from pyspark import SparkContext, SparkConf

In [157]:
textFile = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/quiz2.txt")
type(textFile)

#textFile.collect()
rdd_flat= textFile.flatMap(lambda x:x.split(' '))
rdd_key_val = rdd_flat.map(lambda x: (x, 1))
rdd_reduce = rdd_key_val.reduceByKey(lambda x, y: x + y)
print(rdd_reduce.collect())

[('this', 1), ('mango', 1), ('cat', 1), ('ant', 2), ('laptop', 1), ('chair', 1), ('mobile', 1), ('am', 1), ('este', 1), ('mi', 1), ('ejemplo', 1), ('company', 1), ('animal', 1), ('dog', 1), ('mic', 1), ('seitch', 1), ('charger', 1), ('cover', 1), ('amanda', 1), ('any', 1), ('alarm', 1), ('es', 1)]


11. Count y CountByValue

Ambas son acciones que se aplican sobre __RDD__
Count devuelve la cantidad de elementos dentro de un __rdd__
CountByValue devuelve la cantidad de veces que una elemento aparece en un __rdd__

In [171]:
print(textFile.count())
print(rdd_key_val.count())
print(rdd_flat.countByValue())


5
23
defaultdict(<class 'int'>, {'this': 1, 'mango': 1, 'company': 1, 'animal': 1, 'cat': 1, 'dog': 1, 'ant': 2, 'mic': 1, 'laptop': 1, 'chair': 1, 'seitch': 1, 'mobile': 1, 'am': 1, 'charger': 1, 'cover': 1, 'amanda': 1, 'any': 1, 'alarm': 1, 'este': 1, 'es': 1, 'mi': 1, 'ejemplo': 1})


In [176]:
for key, value in rdd_flat.countByValue().items():
    if value > 1:
        print (key, ':', value)

ant : 2


12. Rdd saveAsTextFile

Se usa para guardar un __rdd__ como un archivo de texto. POr defecto un __rdd__ está compuesto por dos particiones, asique __vamos a ver dos archivos__ con el 50% de los datos cada uno

In [179]:
print(rdd_flat.getNumPartitions())
rdd_flat.saveAsTextFile("../data/miRDDTextFile")

13. Aumentar numero de particiones

Se puede usar __repartition__ para aumentar o __coalesce__ para disminuirlo.

In [183]:
textFile = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/quiz2.txt")


rdd_flat = textFile.flatMap(lambda x:x.split(' '))
print(rdd_flat.getNumPartitions())

rdd_flat_4 = rdd_flat.repartition(4)
rdd_flat_4.saveAsTextFile("../data/miPartition")
print(rdd_flat_4.getNumPartitions())


2
MapPartitionsRDD[242] at coalesce at NativeMethodAccessorImpl.java:0
2
